# Surprise Test_2 Question_2 





In [2]:
import numpy as np
np.set_printoptions(suppress=True)
E=210e6 #kN/m^2
mu=0.3
t=0.025 #m
w=3000 #kN/m^2

#### Stiffness Matrix Of Triangular Elements 

In [4]:
##coordinates of nodes 
Coords=np.array([[0,0],[0.5,0],[0.5,0.25],[0,0.25],[0.25,0.125]])

#nodes of each triangular element in anticlockwise fashion
Conec=np.array([[1,5,4],[5,3,4],[5,2,3],[1,2,5]])-1     

#number of traingular elements
nElement=Conec.shape[0]

#total number of DOfs
ndof=Coords.shape[0]*2

#dofs corresponding to each node
dofs=np.array([(1,2),(3,4),(5,6),(7,8),(9,10)])-1

#packaging the DOFs as seqence of known DOFs and Unknown Dofs.. dof_seq=[U_unknown U_known]
dof_seq=np.array([1,2,7,8,3,4,5,6,9,10])-1

#total number of free dof's
free_dof=6



In [9]:
##ElementCords contains the coordinates of each node of triangular element
## a typical item represents (x,y) of ith, jth and mth node

ElementCords=np.ndarray((nElement, 3,2))  #Order imp: a typical item represents (x,y) of three nodes of that triangular element in anticlockwise
for i in range(Conec.shape[0]):
    for j in range(Conec.shape[1]):
        ElementCords[i,j,0]=Coords[Conec[i][j]][0]
        ElementCords[i,j,1]=Coords[Conec[i][j]][1]

In [10]:
print(ElementCords)

[[[0.    0.   ]
  [0.25  0.125]
  [0.    0.25 ]]

 [[0.25  0.125]
  [0.5   0.25 ]
  [0.    0.25 ]]

 [[0.25  0.125]
  [0.5   0.   ]
  [0.5   0.25 ]]

 [[0.    0.   ]
  [0.5   0.   ]
  [0.25  0.125]]]


### Calculating the individual element stiffness
- LinearTriangleElementStiffness: (LTES)

In [12]:

def LTES(E,mu,t,p,EleNumber=1):
    
    """LinearTriangleElementStiffnes: E=Elasticity modulus, mu=poissions ratio, t=thickness, 
                                    p=[0 for plane stress, 1 for plane strain]
                                    EleNumber= element number for which the stifness is to be calcuated"""
    
    xi,yi,xj,yj,xm,ym=ElementCords[i].flatten()
    A = (xi*(yj-ym)+ xj*(ym-yi) + xm*(yi-yj))/2
    bi = yj-ym
    bj = ym-yi
    bm = yi-yj
    gi = xm-xj
    gj = xi-xm
    gm = xj-xi
    B=np.asarray([[bi, 0 ,bj, 0, bm, 0],
                  [0, gi, 0, gj, 0 ,gm],
                  [gi,bi,gj,bj,gm,bm]])

    B=B*(1/(2*A))
    if p==1:    #plane stress condition
        D=(E/(1-mu*mu))*np.asarray([[1,mu,0],[mu,1,0],[0,0,(1-mu)/2]])
    elif p==2: #plane strain condition
        D=(E/((1+mu)*(1-2*mu)))*np.asarray([[1-mu,mu,0],[mu,1-mu,0],[0,0,(1-2*mu)/2]])
    
    return t*A*np.dot(B.T,np.dot(D,B))
    

In [14]:
#calculate individual element stiffness matrix and store it in single array Km
Km=np.ndarray((nElement,6,6))
for i in range(nElement):
    Km[i]=LTES(E,mu,t,1,EleNumber=i)
print(f'10e4*\n {np.round(Km/10000,2)}')

10e4*
 [[[ 173.08   93.75 -144.23 -100.96  -28.85    7.21]
  [  93.75  313.7   -86.54  -50.48   -7.21 -263.22]
  [-144.23  -86.54  288.46    0.   -144.23   86.54]
  [-100.96  -50.48    0.    100.96  100.96  -50.48]
  [ -28.85   -7.21 -144.23  100.96  173.08  -93.75]
  [   7.21 -263.22   86.54  -50.48  -93.75  313.7 ]]

 [[ 403.85    0.   -201.92 -100.96 -201.92  100.96]
  [   0.   1153.85  -86.54 -576.92   86.54 -576.92]
  [-201.92  -86.54  173.08   93.75   28.85   -7.21]
  [-100.96 -576.92   93.75  313.7     7.21  263.22]
  [-201.92   86.54   28.85    7.21  173.08  -93.75]
  [ 100.96 -576.92   -7.21  263.22  -93.75  313.7 ]]

 [[ 288.46    0.   -144.23   86.54 -144.23  -86.54]
  [   0.    100.96  100.96  -50.48 -100.96  -50.48]
  [-144.23  100.96  173.08  -93.75  -28.85   -7.21]
  [  86.54  -50.48  -93.75  313.7     7.21 -263.22]
  [-144.23 -100.96  -28.85    7.21  173.08   93.75]
  [ -86.54  -50.48   -7.21 -263.22   93.75  313.7 ]]

 [[ 173.08   93.75   28.85   -7.21 -201.92  -86.54]

### Global Stiffness Matrix 
- LinearTriangleAssemble: (LTA)

In [15]:
K = np.zeros((8, 8))
#Function calcuates GLOBAL Stiffness matrix

def LTA(K :np.ndarray, k, nodes):
    i,j,m=nodes
    
    K[2*i-1-1, 2*i-1-1] = K[2*i-1-1, 2*i-1-1] + k[1-1, 1-1]
    K[2*i-1-1, 2*i-1] = K[2*i-1-1,  2*i-1] + k[1-1, 2-1]
    K[2*i-1-1, 2*j-1-1] = K[2*i-1-1,  2*j-1-1] + k[1-1, 3-1]
    K[2*i-1-1,  2*j-1] = K[2*i-1-1,  2*j-1] + k[1-1, 4-1]
    K[2*i-1-1,  2*m-1-1] = K[2*i-1-1,  2*m-1-1] + k[1-1, 5-1]
    K[2*i-1-1,  2*m-1] = K[2*i-1-1,  2*m-1] + k[1-1, 6-1]
    K[2*i-1, 2*i-1-1] = K[2*i-1, 2*i-1-1] + k[2-1, 1-1]
    K[2*i-1, 2*i-1] = K[2*i-1, 2*i-1] + k[2-1, 2-1]
    K[2*i-1, 2*j-1-1] = K[2*i-1, 2*j-1-1] + k[2-1, 3-1]
    K[2*i-1, 2*j-1] = K[2*i-1, 2*j-1] + k[2-1, 4-1]
    K[2*i-1, 2*m-1-1] = K[2*i-1, 2*m-1-1] + k[2-1, 5-1]
    K[2*i-1, 2*m-1] = K[2*i-1, 2*m-1] + k[2-1, 6-1]
    K[2*j-1-1, 2*i-1-1] = K[2*j-1-1,  2*i-1-1] + k[3-1, 1-1]
    K[2*j-1-1,  2*i-1] = K[2*j-1-1,  2*i-1] + k[3-1, 2-1]
    K[2*j-1-1,  2*j-1-1] = K[2*j-1-1,  2*j-1-1] + k[3-1, 3-1]
    K[2*j-1-1,  2*j-1] = K[2*j-1-1,  2*j-1] + k[3-1, 4-1]
    K[2*j-1-1,  2*m-1-1] = K[2*j-1-1,  2*m-1-1] + k[3-1, 5-1]
    K[2*j-1-1,  2*m-1] = K[2*j-1-1, 2*m-1] + k[3-1, 6-1]
    K[2*j-1, 2*i-1-1] = K[2*j-1, 2*i-1-1] + k[4-1, 1-1]
    K[2*j-1, 2*i-1] = K[2*j-1, 2*i-1] + k[4-1, 2-1]
    K[2*j-1, 2*j-1-1] = K[2*j-1, 2*j-1-1] + k[4-1, 3-1]
    K[2*j-1, 2*j-1] = K[2*j-1, 2*j-1] + k[4-1, 4-1]
    K[2*j-1, 2*m-1-1] = K[2*j-1, 2*m-1-1] + k[4-1, 5-1]
    K[2*j-1, 2*m-1] = K[2*j-1, 2*m-1] + k[4-1, 6-1]
    K[2*m-1-1,  2*i-1-1] = K[2*m-1-1,  2*i-1-1] + k[5-1, 1-1]
    K[2*m-1-1,  2*i-1] = K[2*m-1-1, 2*i-1] + k[5-1, 2-1]
    K[2*m-1-1,  2*j-1-1] = K[2*m-1-1,  2*j-1-1] + k[5-1, 3-1]
    K[2*m-1-1,  2*j-1] = K[2*m-1-1,  2*j-1] + k[5-1, 4-1]
    K[2*m-1-1,  2*m-1-1] = K[2*m-1-1,  2*m-1-1] + k[5-1, 5-1]
    K[2*m-1-1, 2*m-1] = K[2*m-1-1,  2*m-1] + k[5-1, 6-1]
    K[2*m-1, 2*i-1-1] = K[2*m-1, 2*i-1-1] + k[6-1, 1-1]
    K[2*m-1, 2*i-1] = K[2*m-1, 2*i-1] + k[6-1, 2-1]
    K[2*m-1, 2*j-1-1] = K[2*m-1, 2*j-1-1] + k[6-1, 3-1]
    K[2*m-1, 2*j-1] = K[2*m-1, 2*j-1] + k[6-1, 4-1]
    K[2*m-1, 2*m-1-1] = K[2*m-1, 2*m-1-1] + k[6-1, 5-1]
    K[2*m-1, 2*m-1] = K[2*m-1, 2*m-1] + k[6-1, 6-1]
    
    return K
    

In [16]:
np.set_printoptions(linewidth=np.inf)
KG=np.zeros((ndof,ndof))
## Calcuating the GLOBAL MATRIX using individual matricies
for i in range(nElement):
    nodes=Conec[i]+1
    KG=LTA(KG,Km[i],nodes)
print("GLOBAL MATRIX\n")
print(f'10e4*\n {np.round(KG/10000,4)}')   

GLOBAL MATRIX

10e4*
 [[ 346.1538  187.5      28.8462   -7.2115    0.        0.      -28.8462    7.2115 -346.1538 -187.5   ]
 [ 187.5     627.4038    7.2115  263.2212    0.        0.       -7.2115 -263.2212 -187.5    -627.4038]
 [  28.8462    7.2115  346.1538 -187.5     -28.8462   -7.2115    0.        0.     -346.1538  187.5   ]
 [  -7.2115  263.2212 -187.5     627.4038    7.2115 -263.2212    0.        0.      187.5    -627.4038]
 [   0.        0.      -28.8462    7.2115  346.1538  187.5      28.8462   -7.2115 -346.1538 -187.5   ]
 [   0.        0.       -7.2115 -263.2212  187.5     627.4038    7.2115  263.2212 -187.5    -627.4038]
 [ -28.8462   -7.2115    0.        0.       28.8462    7.2115  346.1538 -187.5    -346.1538  187.5   ]
 [   7.2115 -263.2212    0.        0.       -7.2115  263.2212 -187.5     627.4038  187.5    -627.4038]
 [-346.1538 -187.5    -346.1538  187.5    -346.1538 -187.5    -346.1538  187.5    1384.6154    0.    ]
 [-187.5    -627.4038  187.5    -627.4038 -187.5   

### Boundary Conditions

In [9]:
F=np.zeros((ndof,))
U=np.zeros((ndof,))

In [17]:
#Applying Boundary conditions
U=np.zeros((ndof,1))
F=np.zeros((ndof,1))

#Loading Conditions
F[2,0]=9.375
F[3,0]=0
F[4,0]=9.375
F[5,0]=0
F[8,0]=0
F[9,0]=0


dof_seq=np.array([3,4,5,6,9,10,1,2,7,8])-1  #segregated as per knowns and unknowns [knonws unknowns]
xx=free_dof
#Partitioning the matrices
Up=U[dof_seq[:xx],]
Ux=U[dof_seq[xx:],] #knowns disps
Fp=F[dof_seq[:xx],]
Kpp=KG[np.ix_(dof_seq[:xx],dof_seq[:xx])]


# print(Kpp)

print('\nKnown nodal forces:=\n')
print(Fp)

Up=np.dot(np.linalg.inv(Kpp),Fp)
print('\nUnknown nodal disps:=\n')
print(Up)





Known nodal forces:=

[[9.375]
 [0.   ]
 [9.375]
 [0.   ]
 [0.   ]
 [0.   ]]

Unknown nodal disps:=

[[ 0.00000693]
 [ 0.00000071]
 [ 0.00000693]
 [-0.00000071]
 [ 0.00000327]
 [ 0.        ]]


In [18]:
#Adding Calculated unknown displacements to the main displacement vector
U[dof_seq[:xx],]=Up

#Calculate force
F=np.dot(KG,U)

print('FORCES VECTOR(kN):=\n')
print(np.round(F,7))

FORCES VECTOR(kN):=

[[-9.375    ]
 [-3.7540043]
 [ 9.375    ]
 [-0.       ]
 [ 9.375    ]
 [ 0.       ]
 [-9.375    ]
 [ 3.7540043]
 [-0.       ]
 [ 0.       ]]


### Post Processing 

In [19]:
##Array which contains displacement(nodal) of a traingular element as rows. total colums equals total memebrs


u=np.ndarray((nElement,6))
for p in range(nElement):
    xi=dofs[Conec[p][0]][0]
    yi=dofs[Conec[p][0]][1]
    
    xj=dofs[Conec[p][1]][0]
    yj=dofs[Conec[p][1]][1]
    
    xm=dofs[Conec[p][2]][0]
    ym=dofs[Conec[p][2]][1]
    
    u[p]=U[[xi,yi,xj,yj,xm,ym],].T
    

In [20]:
# Each row corresponds to displacements of a node of a particular trinagular element
#since each trinagular element has 3 nodes, therefore each row has 6 displacemnts 
#total rows corresponds to total number of memebrs
print(u*10e5)


[[ 0.          0.          3.27082221  0.          0.          0.        ]
 [ 3.27082221  0.          6.92834261 -0.71390436  0.          0.        ]
 [ 3.27082221  0.          6.92834261  0.71390436  6.92834261 -0.71390436]
 [ 0.          0.          6.92834261  0.71390436  3.27082221  0.        ]]


# Element stresses
- LinearTriangleElementStresses: (LTEStress)

In [22]:

def LTEStress(E,mu,p=1,EleNumber=1):
    xi,yi,xj,yj,xm,ym=ElementCords[i].flatten()
    uu=u[EleNumber]
    
    A = (xi*(yj-ym)+ xj*(ym-yi) + xm*(yi-yj))/2
    bi = yj-ym
    bj = ym-yi
    bm = yi-yj
    gi = xm-xj
    gj = xi-xm
    gm = xj-xi
    B=np.asarray([[bi, 0 ,bj, 0, bm, 0],
                  [0, gi, 0, gj, 0 ,gm],
                  [gi,bi,gj,bj,gm,bm]])

    B=B*(1/(2*A))
    
    if p==1:    #plane stress condition
        D=(E/(1-mu*mu))*np.asarray([[1,mu,0],
                                    [mu,1,0],
                                    [0,0,(1-mu)/2]])
        
    elif p==2: #plane strain condition
        D=(E/((1+mu)*(1-2*mu)))*np.asarray([[1-mu,mu,0],
                                            [mu,1-mu,0],
                                            [0,0,(1-2*mu)/2]])

    return np.dot(D,np.dot(B,uu)).reshape(np.dot(D,np.dot(B,uu)).shape[0],1)

# Function to calculate Principle Stresses
- LinearTriangleElementPStresses: (LTEPStress)

In [24]:
# FUNCTION TO CALCUATE PRINCIPLE STRESSSES
def LTEPStress(sigma):
    R = (sigma[0] + sigma[1])/2
    Q = ((sigma[0] - sigma[1])/2)**2 + sigma[2]*sigma[2]
    M = 2*sigma[2]/(sigma[0] - sigma[1])
    s1 = R + np.sqrt(Q)
    s2 = R - np.sqrt(Q)
    theta = (np.arctan(M)/2)*180/np.pi
    return np.asarray([s1 , s2 , theta])

In [28]:

#sigma is an array having stesses as its items
#for a typical item, it correspongs to sigma_x,sigma_y, tau_xy
#number of items correponds to number of traingular elements

sigma=np.ndarray((nElement,3,1))
for i in range(nElement):
    sigma[i]=LTEStress(E,mu,EleNumber=i)
# print(sigma*1000)    

#### Stresses = [Sigma x Sigma y Tau xy] 

In [29]:
for i in range(nElement):
    print(f'\nStress:ELEMENT{i+1}\n\n {sigma[i]}')


Stress:ELEMENT1

 [[3019.22050187]
 [ 905.76615056]
 [   0.        ]]

Stress:ELEMENT2

 [[3000.        ]
 [ 300.3203417 ]
 [   9.61025093]]

Stress:ELEMENT3

 [[2980.77949813]
 [-305.12546716]
 [  -0.        ]]

Stress:ELEMENT4

 [[3000.        ]
 [ 300.3203417 ]
 [  -9.61025093]]


In [33]:
#principleStress is an array having principal stresses and theta items

PrincipleStress=np.ndarray(sigma.shape)
for i in range(nElement):
    PrincipleStress[i]= LTEPStress(sigma[i])
# print(PrincipleStress)

####  Principal Stresses:= [Sigma x Sigma y Theta ] 

In [35]:
for i in range(nElement):
    print(f'\nPrincipal Stress:=ELEMENT{i+1}\n\n {PrincipleStress[i]}')


Principal Stress:=ELEMENT1

 [[3019.22050187]
 [ 905.76615056]
 [   0.        ]]

Principal Stress:=ELEMENT2

 [[3000.03420989]
 [ 300.2861318 ]
 [   0.20395661]]

Principal Stress:=ELEMENT3

 [[2980.77949813]
 [-305.12546716]
 [  -0.        ]]

Principal Stress:=ELEMENT4

 [[3000.03420989]
 [ 300.2861318 ]
 [  -0.20395661]]
